In [ ]:
import numpy as np
import pandas as pd 
import os
import glob
import time
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, CSVLogger, EarlyStopping
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score, confusion_matrix


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
os.chdir('/content/drive/MyDrive/UCF_PROCESSED_DATA/training')

In [ ]:
# Required Parameters

train_path = "/content/drive/MyDrive/UCF_PROCESSED_DATA/training/"            # Training Path
test_path = '/content/drive/MyDrive/UCF_PROCESSED_DATA/testing/'             # Testing Path
no_of_frames = 2200                                                                        # Model Selection Choice
epochs = 3                                                                    # Number of epochs
batch_size = 32                                                                # Batch Size
n_classes = 6                                                               # Number of Classes
patience = 2                                                                    # Patience for EarlyStopping
stime = int(time.time())                                                        # Defining Starting Time
categories = os.listdir(train_path)                                             # Name of each Class/Category

In [ ]:
categories.sort()
print(categories)

['basketball', 'biking', 'diving', 'golf_swing', 'horse_riding', 'walking']


In [ ]:
from tensorflow.keras.applications.resnet import ResNet50, preprocess_input
base_model = ResNet50(weights = 'imagenet', include_top = False, input_shape = (224, 224, 3))

In [ ]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation = 'relu')(x)
x = Dropout(0.5)(x)
# x = Dense(512, activation = 'relu')(x)
# x = Dense(256, activation = 'relu')(x)
preds = Dense(n_classes, activation = 'softmax')(x)

In [ ]:
model = Model(inputs = base_model.input, outputs = preds)

In [ ]:
# Printing the names of each layer
for i, layer in enumerate(model.layers):
    print(i, layer.name)

0 input_3
1 conv1_pad
2 conv1_conv
3 pool1_pad
4 pool1_pool
5 conv2_block1_preact_bn
6 conv2_block1_preact_relu
7 conv2_block1_1_conv
8 conv2_block1_1_bn
9 conv2_block1_1_relu
10 conv2_block1_2_pad
11 conv2_block1_2_conv
12 conv2_block1_2_bn
13 conv2_block1_2_relu
14 conv2_block1_0_conv
15 conv2_block1_3_conv
16 conv2_block1_out
17 conv2_block2_preact_bn
18 conv2_block2_preact_relu
19 conv2_block2_1_conv
20 conv2_block2_1_bn
21 conv2_block2_1_relu
22 conv2_block2_2_pad
23 conv2_block2_2_conv
24 conv2_block2_2_bn
25 conv2_block2_2_relu
26 conv2_block2_3_conv
27 conv2_block2_out
28 conv2_block3_preact_bn
29 conv2_block3_preact_relu
30 conv2_block3_1_conv
31 conv2_block3_1_bn
32 conv2_block3_1_relu
33 conv2_block3_2_pad
34 conv2_block3_2_conv
35 conv2_block3_2_bn
36 conv2_block3_2_relu
37 max_pooling2d_6
38 conv2_block3_3_conv
39 conv2_block3_out
40 conv3_block1_preact_bn
41 conv3_block1_preact_relu
42 conv3_block1_1_conv
43 conv3_block1_1_bn
44 conv3_block1_1_relu
45 conv3_block1_2_pad
4

In [ ]:
print(model.summary())

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_3[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                            

In [ ]:
# Setting each layer as trainable
for layer in model.layers:
    layer.trainable = True

In [ ]:
# Defining Image Data Generators
train_datagenerator = ImageDataGenerator(preprocessing_function = preprocess_input,
                                         validation_split = 0.2)

test_datagenerator = ImageDataGenerator(preprocessing_function = preprocess_input)

train_generator = train_datagenerator.flow_from_directory(train_path,
                                                          target_size = (224, 224),
                                                          color_mode = 'rgb',
                                                          batch_size = batch_size,
                                                          class_mode = 'categorical',
                                                          shuffle = True)

validation_generator = train_datagenerator.flow_from_directory(train_path,
                                                               target_size = (224, 224),
                                                               color_mode = 'rgb',
                                                               batch_size = batch_size,
                                                               class_mode = 'categorical',
                                                               subset = 'validation')

test_generator = test_datagenerator.flow_from_directory(test_path,
                                                        target_size = (224, 224),
                                                        color_mode = 'rgb',
                                                        class_mode = 'categorical')

Found 12310 images belonging to 6 classes.
Found 2462 images belonging to 6 classes.
Found 900 images belonging to 6 classes.


In [ ]:
print(train_generator.class_indices)
print(validation_generator.class_indices)
print(test_generator.class_indices)

{'basketball': 0, 'biking': 1, 'diving': 2, 'golf_swing': 3, 'horse_riding': 4, 'walking': 5}
{'basketball': 0, 'biking': 1, 'diving': 2, 'golf_swing': 3, 'horse_riding': 4, 'walking': 5}
{'basketball': 0, 'biking': 1, 'diving': 2, 'golf_swing': 3, 'horse_riding': 4, 'walking': 5}


In [ ]:
# Compiling the Model
model.compile(optimizer = "Adam",
              loss = "categorical_crossentropy",
              metrics = ["accuracy"])

In [ ]:
# Creating a timestamp directory
try:
    os.mkdir("{}_{}b_{}e".format(stime, batch_size, epochs))
except:
    print("Directory already present...")

In [ ]:
# CSVLogger
filename = "{}_{}b_{}\\file.csv".format(stime, batch_size, epochs)
csv_log = CSVLogger(filename)

In [ ]:
# Early Stopping
early_stopping = EarlyStopping(patience = patience)

In [ ]:
tensorboard = TensorBoard(log_dir = "{}_{}b_{}e\logs".format(stime, batch_size, epochs))


In [ ]:
# Defining Model Checkpoint
checkpoint_name = "{}_{}b_{}e".format(stime, batch_size, epochs)
checkpoint_path = checkpoint_name + "\cp-{epoch:04d}-{accuracy:.4f}a-{loss:.4f}l-{val_accuracy:.4f}va-{val_loss:.4f}vl.h5"
checkpoint_dir = os.path.dirname(checkpoint_path)
model_checkpoint = ModelCheckpoint(checkpoint_path)

In [ ]:
# Training the Model
history = model.fit(train_generator,
                    validation_data = validation_generator,
                    epochs = epochs,
                    callbacks = [model_checkpoint, tensorboard, csv_log, early_stopping])

Epoch 1/3
385/385 [==============================] - 8028s 21s/step - loss: 0.4136 - accuracy: 0.8724 - val_loss: 2.0724 - val_accuracy: 0.7746
Epoch 2/3
385/385 [==============================] - 8062s 21s/step - loss: 0.1314 - accuracy: 0.9595 - val_loss: 0.0869 - val_accuracy: 0.9740
Epoch 3/3
 32/385 [=>............................] - ETA: 1:56:49 - loss: 0.0613 - accuracy: 0.9795

In [ ]:

# Plotting the Graph
model_history = pd.DataFrame(history.history)
model_history.plot()

In [ ]:
history2 = model.evaluate(test_generator)

In [ ]:
# Image Data Generator
test_datagen = ImageDataGenerator(preprocessing_function = preprocess_input)

test_generator = test_datagen.flow_from_directory(test_path,
                                                  target_size = (224, 224),
                                                  color_mode = "rgb",
                                                  shuffle = False,
                                                  class_mode = 'categorical',
                                                  batch_size = 1)

In [ ]:
activities = test_generator.class_indices
print(activities)

In [ ]:
def get_activity(val):
    for key, value in activities.items():
        if val == value:
            return key
    return "Invalid"

In [ ]:

filenames = test_generator.filenames
nb_samples = len(filenames)

In [ ]:
predict = model.predict(test_generator, steps = nb_samples, verbose = 1)

In [ ]:
y_pred = []
for val in predict:
    y_pred.append(get_activity(np.argmax(val)))

y_true = []
for file in filenames:
    y_true.append(file.split("/")[0])

In [ ]:
cm = confusion_matrix(y_true, y_pred)

print(precision_score(y_true, y_pred, average = 'macro',zero_division=1))
print(recall_score(y_true, y_pred, average = 'macro',zero_division=1))
print(f1_score(y_true, y_pred, average = 'macro',zero_division=1))

print(precision_score(y_true, y_pred, average = 'micro',zero_division=1))
print(recall_score(y_true, y_pred, average = 'micro',zero_division=1))
print(f1_score(y_true, y_pred, average = 'micro',zero_division=1))

In [ ]:
# Making a Classification Report
print(classification_report(y_true, y_pred))

dataframe = pd.DataFrame(cm)
inv_dict = {v: k for k, v in activities.items()} 
dataframe = dataframe.rename(index = inv_dict)
dataframe = dataframe.rename(columns = inv_dict)

In [ ]:
# Saving Confusion Matrix in CSV format
dataframe.to_csv("Perfomance Confusion Matrix.csv")